In [1]:
import pickle
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
# 从本地文件加载
with open('../mid_data/CEPCDataset.pkl', 'rb') as f:
    CEPCDataset = pickle.load(f)
print(CEPCDataset.shape)

with open('../mid_data/CEPCTarget.pkl', 'rb') as f:
    CEPCTarget = pickle.load(f)
print(CEPCTarget.shape)

(1650594, 5)
(1650594, 1)


In [3]:
result = CEPCDataset[['dataset', 'A_parent', 'B_child']].copy()

In [4]:
def correlation_feature(A, B):
    return {'corr': np.corrcoef(A, B)[0, 1]}

In [5]:
# 定义特征工程函数列表
feature_functions = [
    correlation_feature,
    # 添加其他特征工程函数
]

In [6]:
def apply_feature_function(row, func):
    """
    应用单个特征工程函数到一行数据。

    参数:
        row (pd.Series): CEPCDataset中的一行数据。
        func (function): 特征工程函数。

    返回:
        dict: 特征工程函数生成的特征。
    """
    try:
        # 假设每个函数接收A和B并返回一个字典
        feature_dict = func(row['A'], row['B'])
        return feature_dict
    except Exception as e:
        print(f"特征函数 {func.__name__} 处理失败: {e}")
        return {}

In [7]:
# 逐个特征工程函数进行处理
for func in feature_functions:
    print(f"正在处理特征工程函数: {func.__name__}")
    
    # 使用多线程并行处理当前特征工程函数
    feature_list = Parallel(n_jobs=-1)(
        delayed(apply_feature_function)(row, func) 
        for index, row in tqdm(CEPCDataset.iterrows(), total=CEPCDataset.shape[0], desc=f"Processing {func.__name__}")
    )
    
    # 将特征列表转换为 DataFrame
    feature_df = pd.DataFrame(feature_list)
    
    # # 确保特征名唯一，添加函数名作为前缀
    # feature_df = feature_df.add_prefix(f"{func.__name__}_")
    
    # 合并当前特征到结果数据框
    result = pd.concat([result.reset_index(drop=True), feature_df.reset_index(drop=True)], axis=1)
    
    print(f"{func.__name__} 特征工程完成。")

# 确保结果数据框的索引与 CEPCDataset 相同
result.index = CEPCDataset.index

正在处理特征工程函数: correlation_feature


Processing correlation_feature: 100%|██████████| 1650594/1650594 [06:31<00:00, 4220.15it/s]


correlation_feature 特征工程完成。


In [ ]:
# 保存最终的数据框
result.to_pickle('../mid_data/CEPCDataset_with_features.pkl')
print('所有特征工程完成，数据已保存。')